# 02 — Workflows vs Agentes

**Objetivo**: Comparar el patron workflow (lineal, predecible) con el patron agente (dinamico, autonomo) y entender cuando usar cada uno.

## Contenido
1. Workflow lineal: translate → summarize → format
2. Agente dinamico con tool calling
3. Comparacion side-by-side
4. Taxonomia de Anthropic (6 patrones)
5. Arbol de decision

In [ ]:
import os
import json
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()
MODEL = "gpt-5-mini"

print("=" * 60)
print("WORKFLOWS vs AGENTES")
print("=" * 60)

## 1. Workflow Lineal (Prompt Chaining)

Un workflow es una secuencia fija de pasos. Cada paso recibe la salida del anterior.

```
Input → [Traducir] → [Resumir] → [Formatear] → Output
```

Ventajas: predecible, debuggeable, costo fijo.
Desventaja: no se adapta a inputs variados.

In [ ]:
# ============================================================
# WORKFLOW LINEAL: Translate → Summarize → Format
# ============================================================

def workflow_lineal(texto: str) -> dict:
    """
    Pipeline lineal de 3 pasos.

    Args:
        texto: Texto de entrada en cualquier idioma.

    Returns:
        Dict con resultado de cada paso y metricas.
    """
    pasos = []
    total_tokens = 0
    t_start = time.time()

    # Paso 1: Traducir a español
    r1 = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Traduce el siguiente texto al español. Solo devuelve la traduccion."},
            {"role": "user", "content": texto}
        ],
    )
    traduccion = r1.choices[0].message.content
    total_tokens += r1.usage.total_tokens
    pasos.append({"paso": "traducir", "resultado": traduccion})

    # Paso 2: Resumir
    r2 = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Resume el siguiente texto en 2-3 oraciones. Solo devuelve el resumen."},
            {"role": "user", "content": traduccion}
        ],
    )
    resumen = r2.choices[0].message.content
    total_tokens += r2.usage.total_tokens
    pasos.append({"paso": "resumir", "resultado": resumen})

    # Paso 3: Formatear
    r3 = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Formatea el siguiente texto como bullet points en markdown. Solo devuelve los bullet points."},
            {"role": "user", "content": resumen}
        ],
    )
    formateado = r3.choices[0].message.content
    total_tokens += r3.usage.total_tokens
    pasos.append({"paso": "formatear", "resultado": formateado})

    latencia_ms = (time.time() - t_start) * 1000
    costo = total_tokens * 0.375 / 1_000_000  # promedio input/output

    return {
        "tipo": "workflow",
        "pasos": pasos,
        "resultado_final": formateado,
        "metricas": {
            "num_pasos": 3,
            "total_tokens": total_tokens,
            "latencia_ms": round(latencia_ms, 1),
            "costo_usd": round(costo, 6),
        }
    }


texto_prueba = """
Artificial intelligence agents represent a paradigm shift in how we build software.
Instead of hardcoding every decision path, we let a language model decide what to do next.
This introduces both flexibility and unpredictability.
The key challenge is balancing autonomy with control.
"""

resultado_wf = workflow_lineal(texto_prueba)

print("WORKFLOW LINEAL — Resultados:")
for paso in resultado_wf["pasos"]:
    print(f"\n  [{paso['paso'].upper()}]")
    print(f"  {paso['resultado'][:150]}")

print(f"\nMetricas: {resultado_wf['metricas']}")

## 2. Agente Dinamico

El agente recibe el mismo input pero decide autonomamente que pasos tomar.
Puede traducir, resumir, formatear, o hacer algo completamente diferente segun el input.

In [ ]:
# ============================================================
# AGENTE DINAMICO con herramientas
# ============================================================

AGENT_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "traducir",
            "description": "Traduce texto al español.",
            "parameters": {
                "type": "object",
                "properties": {"texto": {"type": "string", "description": "Texto a traducir"}},
                "required": ["texto"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "resumir",
            "description": "Resume texto en 2-3 oraciones.",
            "parameters": {
                "type": "object",
                "properties": {"texto": {"type": "string", "description": "Texto a resumir"}},
                "required": ["texto"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "formatear_bullets",
            "description": "Formatea texto como bullet points en markdown.",
            "parameters": {
                "type": "object",
                "properties": {"texto": {"type": "string", "description": "Texto a formatear"}},
                "required": ["texto"]
            }
        }
    },
]

def ejecutar_tool(name: str, args: dict) -> str:
    """Ejecuta herramientas del agente via LLM."""
    prompts = {
        "traducir": "Traduce al español. Solo la traduccion:",
        "resumir": "Resume en 2-3 oraciones. Solo el resumen:",
        "formatear_bullets": "Formatea como bullet points markdown. Solo los bullets:",
    }
    r = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompts[name]},
            {"role": "user", "content": args["texto"]}
        ],
    )
    return r.choices[0].message.content, r.usage.total_tokens


def agente_dinamico(texto: str, max_steps: int = 6) -> dict:
    """
    Agente que decide autonomamente que herramientas usar.

    Args:
        texto: Input del usuario.
        max_steps: Maximo de iteraciones.

    Returns:
        Dict con resultado y metricas.
    """
    messages = [
        {"role": "system", "content": "Eres un asistente. Procesa el texto del usuario: traducelo al español, resumelo, y formatealo como bullets. Usa las herramientas disponibles. Responde en español."},
        {"role": "user", "content": texto},
    ]

    pasos = []
    total_tokens = 0
    t_start = time.time()

    for step in range(max_steps):
        response = client.chat.completions.create(
            model=MODEL, messages=messages, tools=AGENT_TOOLS, tool_choice="auto",
        )
        msg = response.choices[0].message
        total_tokens += response.usage.total_tokens

        if not msg.tool_calls:
            pasos.append({"paso": f"respuesta_final", "resultado": msg.content})
            break

        messages.append(msg)
        for tc in msg.tool_calls:
            fn_name = tc.function.name
            fn_args = json.loads(tc.function.arguments)
            resultado_tool, tokens_tool = ejecutar_tool(fn_name, fn_args)
            total_tokens += tokens_tool
            pasos.append({"paso": fn_name, "resultado": resultado_tool[:200]})
            messages.append({"role": "tool", "tool_call_id": tc.id, "content": resultado_tool})

    latencia_ms = (time.time() - t_start) * 1000
    costo = total_tokens * 0.375 / 1_000_000

    return {
        "tipo": "agente",
        "pasos": pasos,
        "resultado_final": pasos[-1]["resultado"] if pasos else "Sin resultado",
        "metricas": {
            "num_pasos": len(pasos),
            "total_tokens": total_tokens,
            "latencia_ms": round(latencia_ms, 1),
            "costo_usd": round(costo, 6),
        }
    }


resultado_ag = agente_dinamico(texto_prueba)

print("AGENTE DINAMICO — Resultados:")
for paso in resultado_ag["pasos"]:
    print(f"\n  [{paso['paso'].upper()}]")
    print(f"  {paso['resultado'][:150]}")

print(f"\nMetricas: {resultado_ag['metricas']}")

## 3. Comparacion Side-by-Side

In [ ]:
import pandas as pd

comparacion = pd.DataFrame([
    {
        "Tipo": r["tipo"],
        "Pasos": r["metricas"]["num_pasos"],
        "Tokens": r["metricas"]["total_tokens"],
        "Latencia (ms)": r["metricas"]["latencia_ms"],
        "Costo (USD)": f"${r['metricas']['costo_usd']:.6f}",
        "Predecible": "Si" if r["tipo"] == "workflow" else "No",
    }
    for r in [resultado_wf, resultado_ag]
])

print("=" * 60)
print("COMPARACION: WORKFLOW vs AGENTE")
print("=" * 60)
print(comparacion.to_string(index=False))

## 4. Taxonomia de Patrones (Anthropic)

Anthropic define 6 patrones de sistemas con LLMs, ordenados de menor a mayor autonomia:

| Patron | Descripcion | Autonomia | Ejemplo |
|--------|-------------|-----------|--------|
| **Prompt Chaining** | Pasos fijos secuenciales | Baja | Traducir → Resumir → Formatear |
| **Routing** | Clasificar input y dirigir a especialista | Baja-Media | Ticket → soporte tecnico o ventas |
| **Parallelization** | Ejecutar multiples tareas simultaneas | Media | Analizar sentimiento + extraer entidades |
| **Orchestrator-Workers** | Un LLM descompone y delega | Media-Alta | "Investiga X" → buscar, leer, sintetizar |
| **Evaluator-Optimizer** | Generar + evaluar + mejorar | Alta | Generar codigo → testear → corregir |
| **Autonomous Agent** | Loop abierto con herramientas | Muy Alta | Agente de investigacion autonomo |

### Regla practica
> **Usa workflows cuando el camino es conocido. Usa agentes cuando el camino depende del input.**

## 5. Arbol de Decision

```
¿El flujo es siempre el mismo?
├── SI → Workflow (Prompt Chaining)
│   ¿Hay pasos paralelizables?
│   ├── SI → Parallelization
│   └── NO → Chaining puro
│
└── NO → ¿Cuantas rutas posibles?
    ├── Pocas (2-5) → Routing
    └── Muchas/Desconocidas → ¿Necesita auto-correccion?
        ├── SI → Evaluator-Optimizer
        └── NO → ¿Subtareas independientes?
            ├── SI → Orchestrator-Workers
            └── NO → Autonomous Agent
```

### Takeaways
1. La mayoria de problemas reales se resuelven con **workflows** (70-80% de los casos)
2. Los agentes autonomos son poderosos pero mas caros, lentos e impredecibles
3. Empezar con el patron mas simple y escalar solo si es necesario
4. El costo de un agente es O(n) en numero de pasos; un workflow es O(1) fijo